In [1]:
# Normally, you can just use `pip install changepoynt` but for reproducibility, we install a certain
# commit from the package's GitHub repository.
!pip install git+https://github.com/Lucew/changepoynt.git@8c70dda856d758bcb6da1d2a313aa8cf81d5529b tqdm

  Cloning https://github.com/Lucew/changepoynt.git (to revision 8c70dda856d758bcb6da1d2a313aa8cf81d5529b) to /tmp/pip-req-build-b1rgapn0
  Running command git clone --filter=blob:none --quiet https://github.com/Lucew/changepoynt.git /tmp/pip-req-build-b1rgapn0
  Running command git rev-parse -q --verify 'sha^8c70dda856d758bcb6da1d2a313aa8cf81d5529b'
  Running command git fetch -q https://github.com/Lucew/changepoynt.git 8c70dda856d758bcb6da1d2a313aa8cf81d5529b
  Resolved https://github.com/Lucew/changepoynt.git to commit 8c70dda856d758bcb6da1d2a313aa8cf81d5529b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 65.7 MB/s eta 0:00:00
  Created wheel for changepoynt: filename=changepoynt-0.0.9-py3-none-any.whl size=58865 sha256=2439dfda165721f69d47b243f7809e8bb1722f4fda0bca8205879900f3f7093a
  Stored in directo

In [2]:
import time

import numpy as np
from tqdm import tqdm

from changepoynt.algorithms.sst import SST

In [3]:
SIZE_FACTOR = 200

In [4]:
# simulate a signal that goes from exponential decline into a sine wave
# the signals is only for demonstration purposes and can be replaced by your signal
steady_before = np.ones(20*SIZE_FACTOR)
exp_signal = np.exp(-np.linspace(0, 5, 20*SIZE_FACTOR))
steady_after = np.exp(-5)*np.ones(15)
sine_after = 0.2*np.sin(np.linspace(0, 3*np.pi*10, 30*SIZE_FACTOR))
signal = np.concatenate((steady_before, exp_signal, steady_after, sine_after))
signal += 0.02*np.random.randn(signal.shape[0])  # add some minor noise

In [5]:
# create each of the detectors
# We enable the fast Hankel matrix product by: use_fast_hankel=True
# In order to save time, we only compute the change score at ever 50th sample (scoring_step=50)
window_size = 3*SIZE_FACTOR
scoring_step = 50
FFT_RSVD_SST = SST(window_size, method='rsvd', use_fast_hankel=True, scoring_step=scoring_step)
RSVD_SST = SST(window_size, method='rsvd', scoring_step=scoring_step)
FFT_IKA_SST = SST(window_size, method='ika', use_fast_hankel=True, scoring_step=scoring_step)
IKA_SST = SST(window_size, method='ika', scoring_step=scoring_step)
NAIVE_SVD = SST(window_size, method='naive', scoring_step=scoring_step)

In [6]:
# create an iterable with the detectors
detectors = {f'{NAIVE_SVD=}'.split('=')[0]: NAIVE_SVD, f'{RSVD_SST=}'.split('=')[0]: RSVD_SST, f'{FFT_RSVD_SST=}'.split('=')[0]: FFT_RSVD_SST, f'{IKA_SST=}'.split('=')[0]:IKA_SST, f'{FFT_IKA_SST=}'.split('=')[0]: FFT_IKA_SST}
namelength = max(len(key) for key in detectors.keys())

In [7]:
# go through each of the detectors (and keep the score of the naive method)
# On a Google Colab instance, this takes about 4 minutes
score = np.zeros_like(signal)
durations = {name: 0.0 for name in detectors.keys()}
errors = {name: -1 for name in detectors.keys()}
for name, detector in tqdm(detectors.items(), desc="Running detectors"):

    # we have to trigger the JIT compiler first (otherwise we measure compile time)
    if not name.startswith('NAIVE'):
        _ = detector.transform(signal)

    # apply the detectors
    # as the decompositions are applied for several steps, we already get a balanced time measurement for several
    # decompositions that accounts for processing time variations due to CPU scheduling
    start = time.perf_counter()
    method_score = detector.transform(signal)
    durations[name] = time.perf_counter() - start

    # create the ground truth from the naive method
    if name.startswith('NAIVE'):
        score = method_score

    # compute the error
    assert not np.all(score == 0), 'Naive method has to be the first.'
    errors[name] = np.mean(np.abs(score - method_score))

Running detectors: 100%|██████████| 5/5 [03:17<00:00, 39.43s/it]


In [8]:
# print the results to console
for name in detectors.keys():
    duration = durations[name]
    error = errors[name]
    print(f"{name: <{namelength}} took {duration:.2f} seconds for {signal.shape[0]//scoring_step} samples and N={window_size}.")
    print(f"Mean absolute error to naive SST: {error:.3E}")
    print()

NAIVE_SVD    took 129.42 seconds for 280 samples and N=600.
Mean absolute error to naive SST: 0.000E+00

RSVD_SST     took 20.88 seconds for 280 samples and N=600.
Mean absolute error to naive SST: 4.812E-06

FFT_RSVD_SST took 3.97 seconds for 280 samples and N=600.
Mean absolute error to naive SST: 4.789E-06

IKA_SST      took 8.09 seconds for 280 samples and N=600.
Mean absolute error to naive SST: 1.962E-05

FFT_IKA_SST  took 0.38 seconds for 280 samples and N=600.
Mean absolute error to naive SST: 1.519E-05

